# the randomized rounding algorithm for quantum max cut

In [15]:
using JuMP
using LinearAlgebra
using Random
using SCS

In [16]:
function cycle_graph(n)
    edges = []
    for i in 1:n
        push!(edges, (i, mod1(i+1,n)))
    end
    return edges
end

function star_graph(n)
    edges = []
    for i in 2:n
        push!(edges, (1,i))
    end
    return edges
end


star_graph (generic function with 1 method)

In [17]:
function maxcut_sdp(n, edges)
    model = Model(SCS.Optimizer)
    set_silent(model)

    @variable(model, X[1:n,1:n], PSD)

    for i in 1:n
        @constraint(model, X[i,i] == 1)
    end

    @objective(model, Max,
    sum( (1 - X[i,j]) for (i,j) in edges ) / 2
)


    optimize!(model)
    return value.(X), objective_value(model)
end


maxcut_sdp (generic function with 1 method)

In [18]:
function randomized_rounding(X, edges; trials=200)
    n = size(X,1)
    λ, U = eigen(Symmetric(X))
    V = Diagonal(sqrt.(max.(λ,0))) * U'

    best = 0
    for _ in 1:trials
        r = randn(n)
        s = sign.(V' * r)
        cut = sum(s[i] != s[j] for (i,j) in edges)
        best = max(best, cut)
    end
    return best
end


randomized_rounding (generic function with 1 method)

In [19]:
n = 6
edges = cycle_graph(n)
X, sdp_val = maxcut_sdp(n, edges)
cut = randomized_rounding(X, edges)

println("C6:")
println(" SDP value ≈ ", sdp_val)
println(" Rounded cut = ", cut)
println(" Exact maxcut = ", n)


C6:
 SDP value ≈ 5.999985261363761
 Rounded cut = 6
 Exact maxcut = 6


In [20]:
n = 5
edges = cycle_graph(n)
X, sdp_val = maxcut_sdp(n, edges)
cut = randomized_rounding(X, edges)

println("C5:")
println(" SDP value ≈ ", sdp_val)
println(" Rounded cut = ", cut)
println(" Exact maxcut = ", n-1)


C5:
 SDP value ≈ 4.522542357191514
 Rounded cut = 4
 Exact maxcut = 4


In [21]:
n = 10
edges = star_graph(n)
X, sdp_val = maxcut_sdp(n, edges)
cut = randomized_rounding(X, edges)

println("Star graph:")
println(" SDP value ≈ ", sdp_val)
println(" Rounded cut = ", cut)
println(" Exact maxcut = ", n-1)


Star graph:
 SDP value ≈ 9.000021884556345
 Rounded cut = 9
 Exact maxcut = 9
